# Solar PV in Brazil: an ABM

## Trying to get things more automated: second try

In [ ]:
#pip install plotly
#pip install jupyterthemes
#pip install feather-format

In [2]:
import matplotlib.pyplot as plt    
import random
import feather

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import simpy
import numpy as np
import pandas as pd
import math as math

def run_simulation(fALPHA, fBETA, fN_of_firms, finitial_tc_upper, finitial_tc_lower, finitial_cc_upper, 
                   finitial_cc_lower, finitial_projects_total, finitial_lc_mean, finitial_lc_variance, 
                   fOKshare, fSIM_TIME, fRANDOM_SEED, fNUMBER_OF_REPETITIONS, fRANDOMIZE):
    
    '''
    This prevents the simulation from crashing
    '''
    
    global ALPHA
    global BETA
    global N_of_firms
    global initial_tc_upper
    global initial_tc_lower
    global initial_cc_upper
    global initial_cc_lower
    global initial_projects_total
    global initial_lc_mean
    global initial_lc_variance
    global RS
    global SIM_TIME
    global RANDOM_SEED
    global NUMBER_OF_REPETITIONS
    global OKshare
    
    '''
    We must tell the simulation that the parameters are the function's inputs
    '''
    
   #Parameter_in_the_smiulation = function_input
    ALPHA = fALPHA
    BETA = fBETA
    N_of_firms = fN_of_firms
    initial_tc_upper = finitial_tc_upper
    initial_tc_lower = finitial_tc_lower
    initial_cc_upper = finitial_cc_upper
    initial_cc_lower = finitial_cc_lower
    initial_projects_total = finitial_projects_total
    initial_lc_mean = finitial_lc_mean
    initial_lc_variance = finitial_lc_variance
    SIM_TIME = fSIM_TIME
    RANDOM_SEED = fRANDOM_SEED
    NUMBER_OF_REPETITIONS = fNUMBER_OF_REPETITIONS
    RANDOMIZE = fRANDOMIZE
    OKshare = fOKshare
       
    global endDF
    endDF = pd.DataFrame() #we have to set a initial main dataframe
    
    for rep in range (RANDOM_SEED, RANDOM_SEED + NUMBER_OF_REPETITIONS): #the +1 produces 1 repetition
        random.seed(rep) #in order to have pseudo-random numbers, allowing greater replicability
        global SIM
        SIM = rep
        print(round(SIM/NUMBER_OF_REPETITIONS, 3), end = '...')
        
        for j in (0, 0.25, 0.5, 0.75, 1):
            RS = j
            env = simpy.Environment() #starts the environment in each iteration
            env.process(outside_firm(env)) #starts the outside_firm process

            global Df
            Df = pd.DataFrame() # starts the result dataframe for each repetition
          
            projects_dictionary = {0: 22,
                                  1: 11,
                                  2: 9,
                                  3: 7,
                                  4: 5,
                                  5: 5,
                                  6: 3,
                                  7: 3,
                                  8: 2,
                                  9: 2,
                                  10: 2,
                                  11: 2,
                                  12: 2,
                                  13 : 1, 14 : 1, 15 : 1, 16 : 1, 17 : 1, 18 : 1, 19 : 1, 20 : 1, 21 : 1, 22 : 1, 23 : 1,
                                   24 : 1, 25 : 1, 26 : 1, 27 : 1, 28 : 1, 29 : 1}

            for i in range(N_of_firms): #This starts us with the number of firms
                if RANDOMIZE is True:
                    env.process(firm(env, 'Firm %d' % i, 0, 0, random.uniform(initial_tc_lower, initial_tc_upper), 
                                     random.uniform(initial_cc_lower, initial_cc_upper),
                                     math.ceil(initial_projects_total/(i+1)),
                                     math.ceil(initial_projects_total/(i+1)), 0, 0, 0))
                    #with this we have a geometric progression of projects, similar to our case

                    """elif SEMI_RANDOM is True: #TBD
                    initial_projects_total = random
                    env.process(firm(env, 'Firm %d' % i, 0, 0, random.uniform(initial_tc_lower, initial_tc_upper),
                                     random.uniform(initial_cc_lower, initial_cc_upper), initial_projects_total,
                                     initial_projects_total, 0, 0, 0)) 
                    #we produce N_of_firms firms with the previous parameters"""

                else:
                    env.process(firm(env, 'Firm %d' % i, 0, 0, random.uniform(initial_tc_lower, initial_tc_upper),
                                     random.uniform(initial_cc_lower, initial_cc_upper), projects_dictionary.get(i),
                                     projects_dictionary.get(i), 0, 0, 0))                    
            env.run(until=SIM_TIME) # we then run the simulation

            endDF = pd.concat([endDF, Df])
        
    
    endDF = endDF.rename(columns={0: "step", 1: "cc", 2: "tc", 3: "lc", 4: "# of BNDES financed projects", 
                                  5: "# of other banks financed projects", 6: "# of rounds without access to finance", 
                                  7: "# of projects yet to be financed", 8: "name", 9: "run",
                                 10: "internalisation_index", 11: 'rs'})
    
    return print('DONE')

In [3]:
'''
        --- For some reason, this cell MUST be ran after the previous cell. If they are merged, errors arise from 
        the simulation
Course of action:
1- Firms attempt to finance their plants
2- Firms attempt to enhance their capabilities
3- BNDES analyses what happened and decides to lower, raise or maintain its requirements
4- Rinse and repeat
'''


def outside_firm(env):
    global LC
    global SIM_TIME
    global OKshare
    global PROJECTS_FINANCED_BNDES
    global INITIAL_NUMBER_OF_PROJECTS
    global SUM_OF_CC_t1
    global SUM_OF_TC_t1
    global SUM_OF_CC
    global SUM_OF_TC
    while True:
        if (env.now >= 1):
            global RS
            rs= RS
            SUM_OF_CC=SUM_OF_CC_t1
            SUM_OF_TC=SUM_OF_TC_t1
            numerator = PROJECTS_FINANCED_BNDES
            denominator = INITIAL_NUMBER_OF_PROJECTS
            Delta_lc = (numerator/denominator - OKshare)*(env.now/SIM_TIME)
            _LC = LC
            _LC *= (1+rs*Delta_lc)
            LC = _LC if (_LC<=1) else 0.99999999
            SUM_OF_CC_t1 = 0
            SUM_OF_TC_t1 = 0
        else:
            LC = np.random.normal(initial_lc_mean, initial_lc_variance) #the initial local content is established
            if LC<0:
                LC=0.00000001
            if LC>1:
                LC=0.99999999
            SUM_OF_CC=0
            SUM_OF_TC=0
            PROJECTS_FINANCED_BNDES=0
            INITIAL_NUMBER_OF_PROJECTS=0
            SUM_OF_CC_t1=0
            SUM_OF_TC_t1=0
        yield env.timeout(1)

def firm(env, name, fBNDES, fbanks, tc, cc, projects_total, projects_remaining, projects_financed_BNDES,
         projects_financed_bank, projects_failed_BNDES):
    
    while True:
        global SUM_OF_CC #put global to avoid problems...
        global SUM_OF_TC #again, put global to avoid problems...
        global LC
        global SIM
        global RS
        global PROJECTS_FINANCED_BNDES
        global INITIAL_NUMBER_OF_PROJECTS
        global SUM_OF_CC_t1
        global SUM_OF_TC_t1
        #attempting to finance power plants
        
        projects_financed_BNDES_tminus1 = projects_financed_BNDES
        if (tc>=1 and cc>=1 and env.now>0):
            fBNDES = np.random.poisson(np.floor(tc)) if projects_remaining>0 else 0
            #number of projects financed by BNDES
            if fBNDES>=1:
                projects_financed_BNDES += fBNDES if fBNDES<projects_remaining else projects_remaining
                projects_remaining -= fBNDES if fBNDES<projects_remaining else projects_remaining
            else:
                projects_failed_BNDES += 1 if projects_remaining>0 else 0
                fbank = np.random.poisson(np.floor(cc)) if projects_remaining>0 else 0
                if fbank>=1:
                    #number of projects financed by another bank
                    projects_financed_bank += fbank if fbank<projects_remaining else projects_remaining
                    projects_remaining -= fbank if fbank<projects_remaining else projects_remaining
        elif (tc>=1 and cc<1 and env.now>0):
            fBNDES = np.random.poisson(np.floor(tc)) if projects_remaining>0 else 0
            if fBNDES>=1:
                projects_financed_BNDES += fBNDES if fBNDES<projects_remaining else projects_remaining
                projects_remaining -= fBNDES if fBNDES<projects_remaining else projects_remaining
            else:
                projects_failed_BNDES += 1 if projects_remaining>0 else 0
        elif (tc<1 and cc>=1 and env.now>0):
            fbank = np.random.poisson(np.floor(cc)) if projects_remaining>0 else 0 
            #number of projects financed by another bank
            projects_failed_BNDES += 1 if projects_remaining>0 else 0
            projects_financed_bank += fbank if fbank<projects_remaining else projects_remaining
            projects_remaining -= fbank if fbank<projects_remaining else projects_remaining
        else:
            projects_failed_BNDES += 1 if projects_remaining>0 else 0
        
        PROJECTS_FINANCED_BNDES += projects_financed_BNDES - projects_financed_BNDES_tminus1
        responsivity = RS 
        # Revision of capabilities
        
        if env.now>0:
            cc_tminus1 = cc
            tc_tminus1 = tc
            alpha = ALPHA #small is internal to the firm CAPITAL IS THE OUTSIDE PARAMETER/VARIABLE
            beta = BETA
            sum_of_cc = SUM_OF_CC
            sum_of_tc = SUM_OF_TC
            EIcc = (cc/(cc+tc))/(sum_of_cc/(sum_of_cc+sum_of_tc)) 
            #efficiency index for the commercial capability
            EItc = (tc/(cc+tc))/(sum_of_tc/(sum_of_cc+sum_of_tc))
            #efficiency index for the technological capability
            tc += random.gammavariate(1, EIcc*beta*((1-LC)**alpha)) 
            # += means: the previous value plus something
            cc += random.gammavariate(1, EItc*beta*((LC)**alpha))
        SUM_OF_CC_t1 += cc
        SUM_OF_TC_t1 += tc
        
        #our firm has done everything it should, 
        #now we have to save what it did, or it will be lost, like tears in the rain
        
        if env.now>0:
            time_t.append(env.now)
            cc_t.append(cc)
            tc_t.append(tc)
            LC_t.append(LC)
            projects_financed_BNDES_t.append(projects_financed_BNDES)
            projects_financed_bank_t.append(projects_financed_bank)
            projects_failed_BNDES_t.append(projects_failed_BNDES)
            projects_remaining_t.append(projects_remaining)
            name_t.append(name)
            SIM_t.append(SIM)
            index_t.append(LC*(projects_financed_BNDES-projects_financed_BNDES_tminus1))
            responsivity_t.append(responsivity)
        else:
            time_t = [env.now]
            DfFirm = pd.DataFrame()
            cc_t = [cc]
            tc_t = [tc]
            LC_t = [LC]
            projects_financed_BNDES_t = [projects_financed_BNDES]
            projects_financed_bank_t = [projects_financed_bank]
            projects_failed_BNDES_t = [projects_failed_BNDES]
            projects_remaining_t = [projects_remaining]
            INITIAL_NUMBER_OF_PROJECTS += projects_remaining
            name_t = [name]
            SIM_t = [SIM]
            index_t = [LC*(projects_financed_BNDES-projects_financed_BNDES_tminus1)]
            responsivity_t = [responsivity]
                           
        if env.now == SIM_TIME-1:
            dict_firm = {0: time_t, 1: cc_t, 2: tc_t, 3: LC_t,4: projects_financed_BNDES_t,5: projects_financed_bank_t,
                         6: projects_failed_BNDES_t,7: projects_remaining_t
                         ,8: name_t,9: SIM_t,10: index_t,11:responsivity_t}
            DfFirm = pd.DataFrame(dict_firm)
            
        
        global Df #we have to indicate that the Df that it is looking is the global Df, not some local Df.
        Df = pd.concat([Df, DfFirm]) #then we concatenate this firm's a_row into the main dataframe.
        
        
        yield env.timeout(1) #until which step should this event occur? with 1 it occurs each step        
         

In [4]:
#pip install jupyternotify
#%load_ext jupyternotify #this package is just to notify when the simulation is done
#%%notify #in order to notify when done, hashtag it if it is not working properly (intended for jupyter, i guess)

run_simulation(2, #alpha
               0.01, #beta
               30, #number of firms
               0.8, # upper initial tc
               0.5, #lower initial tc
               0.5, #upper initial cc
               0, #lower initial cc
               25, #initial number of projects for each firms
               0.3, #mean initial lc
               0.05, #variance of the initial lc
               0.05, #target percentage of projects for BNDES to finance
               60, #simulation time in months
               1, #initial random seed
               250, #number of repetitions to be done
              False) #for randomize purposes (explained below)
endDF

#True means that companies have a geometric progression of cases (good for other cases and stuff)
#False means that each company has the actual observed number projects

0.004...0.008...0.012...

KeyboardInterrupt: 

In [26]:
feather.write_dataframe(endDF, 'endDF05v6.feather')

In [27]:
endDF.to_csv('C:/Users/55219/Google Drive/--Artigo Solar PV in Brasil ABM/endDF05v6.csv', index=False)

In [5]:
endDF = pd.read_csv('C:/Users/55219/Google Drive/--Artigo Solar PV in Brasil ABM/endDF10v6.csv') 

## Plotly graphics

In [6]:
endDFsums = endDF.drop(['name'], axis=1).groupby(['step', 'run', 'rs'], as_index=False).sum()
endDFsums
#for the internalisation index

,step,run,rs,cc,tc,lc,# of BNDES financed projects,# of other banks financed projects,# of rounds without access to finance,# of projects yet to be financed,internalisation_index
0,0,1,0.00,6.846988,19.592024,9.617902,0,0,30,92,0.000000
1,0,1,0.25,6.988863,19.291774,10.145501,0,0,30,92,0.000000
2,0,1,0.50,6.717310,19.189738,7.541264,0,0,30,92,0.000000
3,0,1,0.75,7.258198,18.816698,10.296151,0,0,30,92,0.000000
4,0,1,1.00,7.721869,19.718454,8.217533,0,0,30,92,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
74995,59,250,0.00,9.770875,27.225965,9.578243,9,0,1770,83,0.957824
74996,59,250,0.25,10.188604,29.565505,8.867525,29,0,1633,63,0.295584
74997,59,250,0.50,7.609691,28.957912,6.378105,18,0,1662,74,0.000000
74998,59,250,0.75,8.491263,29.660924,13.131590,38,0,1626,54,0.000000


In [14]:
fig = make_subplots(rows=2, cols=2, specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]], 
                    subplot_titles=("TC and CC", 
                                    "Internalisation index", 
                                    "Projects yet to be financed", 
                                    "projects financed by BNDES"))

dataFrame1 = endDFsums.loc[endDFsums['rs'] == 0.25]
dataFrame1b = endDF.loc[endDF['rs'] == 0.25]
x_variable_to_show1 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show1 = 'internalisation_index'

dataFrame2 = endDFsums.loc[endDFsums['rs'] == 0.5]
dataFrame2b = endDF.loc[endDF['rs'] == 0.5]
x_variable_to_show2 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show2 = 'internalisation_index'

dataFrame3 = endDFsums.loc[endDFsums['rs'] == 0.75]
dataFrame3b = endDF.loc[endDF['rs'] == 0.75]
x_variable_to_show3 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show3 = 'internalisation_index'

dataFrame4 = endDFsums.loc[endDFsums['rs'] == 1]
dataFrame4b = endDF.loc[endDF['rs'] == 1]
x_variable_to_show4 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show4 = 'internalisation_index'

'''
'''

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show1,
           y=dataFrame1.groupby(['step'], as_index=False)[y_variable_to_show1].max()[y_variable_to_show1], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=1, col=1
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show1,
                     y=dataFrame1.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.75)[y_variable_to_show1]),
             secondary_y=False, row=1, col=1)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show1,
                     y=dataFrame1.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.5)[y_variable_to_show1]),
             secondary_y=False, row=1, col=1)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show1,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.25)[y_variable_to_show1]),
             secondary_y=False, row=1, col=1)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show1,
           y=dataFrame1.groupby(['step'], as_index=False)[y_variable_to_show1].min()[y_variable_to_show1], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=1, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame1b.groupby(['step'], as_index=False)['lc'].quantile(.75)['lc']),
              secondary_y=True, row=1, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame1b.groupby(['step'], as_index=False)['lc'].quantile(.5)['lc']),
              secondary_y=True, row=1, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame1b.groupby(['step'], as_index=False)['lc'].quantile(.25)['lc']),
              secondary_y=True, row=1, col=1)

"""
above: row 1 col 1
below: row 1 col 2
"""
fig.add_trace(go.Scatter(name='max', x=x_variable_to_show2,
           y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].max()[y_variable_to_show2], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=1, col=2
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show2,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.75)[y_variable_to_show2]),
             secondary_y=False, row=1, col=2)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show2,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.5)[y_variable_to_show2]),
             secondary_y=False, row=1, col=2)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show2,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.25)[y_variable_to_show2]),
             secondary_y=False, row=1, col=2)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show2,
           y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].min()[y_variable_to_show2], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=1, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame2b.groupby(['step'], as_index=False)['lc'].quantile(.75)['lc']),
              secondary_y=True, row=1, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame2b.groupby(['step'], as_index=False)['lc'].quantile(.5)['lc']),
              secondary_y=True, row=1, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame2b.groupby(['step'], as_index=False)['lc'].quantile(.25)['lc']),
              secondary_y=True, row=1, col=2)

'''
below: row 2 col 1
'''

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show3,
           y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].max()[y_variable_to_show3], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=2, col=1
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show3,
                     y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].quantile(.75)[y_variable_to_show3]),
             secondary_y=False, row=2, col=1)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show3,
                     y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].quantile(.5)[y_variable_to_show3]),
             secondary_y=False, row=2, col=1)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show3,
                     y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].quantile(.25)[y_variable_to_show3]),
             secondary_y=False, row=2, col=1)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show3,
           y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].min()[y_variable_to_show3], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=2, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame3b.groupby(['step'], as_index=False)['lc'].quantile(.75)['lc']),
              secondary_y=True, row=2, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame3b.groupby(['step'], as_index=False)['lc'].quantile(.5)['lc']),
              secondary_y=True, row=2, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame3b.groupby(['step'], as_index=False)['lc'].quantile(.25)['lc']),
              secondary_y=True, row=2, col=1)

'''
below: row 2 col 2
'''

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show4,
           y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].max()[y_variable_to_show4], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=2, col=2
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show4,
                     y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].quantile(.75)[y_variable_to_show4]),
             secondary_y=False, row=2, col=2)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show4,
                     y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].quantile(.5)[y_variable_to_show4]),
             secondary_y=False, row=2, col=2)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show4,
                     y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].quantile(.25)[y_variable_to_show4]),
             secondary_y=False, row=2, col=2)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show4,
           y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].min()[y_variable_to_show4], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=2, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame4b.groupby(['step'], as_index=False)['lc'].quantile(.75)['lc']),
              secondary_y=True, row=2, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame4b.groupby(['step'], as_index=False)['lc'].quantile(.5)['lc']),
              secondary_y=True, row=2, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show1,
           y=dataFrame4b.groupby(['step'], as_index=False)['lc'].quantile(.25)['lc']),
              secondary_y=True, row=2, col=2)

'''
'''

# Change the bar mode
fig.update_layout(barmode='overlay', template="simple_white")
fig.update_yaxes(automargin=True)

In [9]:
fig = make_subplots(rows=2, cols=2, specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]], 
                    subplot_titles=("TC and CC", 
                                    "Internalisation index", 
                                    "Projects yet to be financed", 
                                    "projects financed by BNDES"))

dataFrame1 = endDF.loc[endDF['rs'] == 0.25]
x_variable_to_show1 = 'tc'
y_variable_to_show1 = 'cc'

dataFrame2 = endDFsums.loc[endDFsums['rs'] == 0.25]
x_variable_to_show2 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show2 = 'internalisation_index'

dataFrame3 = endDFsums.loc[endDFsums['rs'] == 0.25]
x_variable_to_show3 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show3 = '# of projects yet to be financed'

dataFrame4 = endDFsums.loc[endDFsums['rs'] == 0.25]
x_variable_to_show4 = dataFrame1.groupby(['step'], as_index=False).max()['step']
y_variable_to_show4 = '# of BNDES financed projects'

'''
'''

fig.add_trace(go.Scatter(name='max',
                         x=dataFrame1.groupby(['step'], as_index=False)[x_variable_to_show1].max()[x_variable_to_show1],
                         y=dataFrame1.groupby(['step'], as_index=False)[y_variable_to_show1].max()[y_variable_to_show1],
                         mode='markers',
                         marker_color=dataFrame1.groupby(['step'], as_index=False).max()['step']), row=1, col=1
)

fig.add_trace(go.Scatter(name='.75 q',
                     x=dataFrame1.groupby(['step'],
                                          as_index=False)[x_variable_to_show1].quantile(.75)[x_variable_to_show1],
                     y=dataFrame1.groupby(['step'],
                                          as_index=False)[y_variable_to_show1].quantile(.75)[y_variable_to_show1],
              mode='markers',
                        marker_color=dataFrame1.groupby(['step'], as_index=False).max()['step']),
              row=1, col=1)
fig.add_trace(go.Scatter(name='.5 q',
                     x=dataFrame1.groupby(['step'],
                                          as_index=False)[x_variable_to_show1].quantile(.5)[x_variable_to_show1],
                     y=dataFrame1.groupby(['step'],
                                          as_index=False)[y_variable_to_show1].quantile(.5)[y_variable_to_show1],
              mode='markers',
                        marker_color=dataFrame1.groupby(['step'], as_index=False).max()['step']),
              row=1, col=1)
fig.add_trace(go.Scatter(name='.25 q',
                     x=dataFrame1.groupby(['step'],
                                          as_index=False)[x_variable_to_show1].quantile(.25)[x_variable_to_show1],
                     y=dataFrame1.groupby(['step'],
                                          as_index=False)[y_variable_to_show1].quantile(.25)[y_variable_to_show1],
              mode='markers',
                        marker_color=dataFrame1.groupby(['step'], as_index=False).max()['step']),
              row=1, col=1)

fig.add_trace(go.Scatter(name='.min',
                     x=dataFrame1.groupby(['step'],
                                          as_index=False)[x_variable_to_show1].min()[x_variable_to_show1],
                     y=dataFrame1.groupby(['step'],
                                          as_index=False)[y_variable_to_show1].min()[y_variable_to_show1],
                        mode='markers',
                        marker_color=dataFrame1.groupby(['step'], as_index=False).max()['step']),
              row=1, col=1)

"""
above: row 1 col 1
below: row 1 col 2
"""
fig.add_trace(go.Scatter(name='max', x=x_variable_to_show2,
           y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].max()[y_variable_to_show2], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=1, col=2
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show2,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.75)[y_variable_to_show2]),
             secondary_y=False, row=1, col=2)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show2,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.5)[y_variable_to_show2]),
             secondary_y=False, row=1, col=2)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show2,
                     y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].quantile(.25)[y_variable_to_show2]),
             secondary_y=False, row=1, col=2)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show2,
           y=dataFrame2.groupby(['step'], as_index=False)[y_variable_to_show2].min()[y_variable_to_show2], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=1, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show2,
           y=dataFrame1.groupby(['step'], as_index=False)['lc'].quantile(.75)['lc']),
              secondary_y=True, row=1, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show2,
           y=dataFrame1.groupby(['step'], as_index=False)['lc'].quantile(.5)['lc']),
              secondary_y=True, row=1, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show2,
           y=dataFrame1.groupby(['step'], as_index=False)['lc'].quantile(.25)['lc']),
              secondary_y=True, row=1, col=2)

'''
below: row 2 col 1
'''

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show3,
           y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].max()[y_variable_to_show3], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=2, col=1
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show3,
                     y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].quantile(.75)[y_variable_to_show3]),
             secondary_y=False, row=2, col=1)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show3,
                     y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].quantile(.5)[y_variable_to_show3]),
             secondary_y=False, row=2, col=1)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show3,
                     y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].quantile(.25)[y_variable_to_show3]),
             secondary_y=False, row=2, col=1)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show3,
           y=dataFrame3.groupby(['step'], as_index=False)[y_variable_to_show3].min()[y_variable_to_show3], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=2, col=1)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show3,
           y=endDF.groupby(['step'], as_index=False)['lc'].min()['lc']),
              secondary_y=True, row=2, col=1)

'''
below: row 2 col 2
'''

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show4,
           y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].max()[y_variable_to_show4], 
               marker=dict(color="crimson", size=3), mode="markers"), secondary_y=False, row=2, col=2
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show4,
                     y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].quantile(.75)[y_variable_to_show4]),
             secondary_y=False, row=2, col=2)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show4,
                     y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].quantile(.5)[y_variable_to_show4]),
             secondary_y=False, row=2, col=2)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show4,
                     y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].quantile(.25)[y_variable_to_show4]),
             secondary_y=False, row=2, col=2)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show4,
           y=dataFrame4.groupby(['step'], as_index=False)[y_variable_to_show4].min()[y_variable_to_show4], 
               marker=dict(color="blue", size=3), mode="markers"),
              secondary_y=False, row=2, col=2)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show4,
           y=endDF.groupby(['step'], as_index=False)['lc'].mean()['lc']),
              secondary_y=True, row=2, col=2)

'''
'''

# Change the bar mode
fig.update_layout(barmode='overlay', template="simple_white")
fig.update_yaxes(automargin=True)

In [ ]:
endDFmax = endDF.drop(['name'], axis=1).groupby(['step', 'run'], as_index=False).max()
endDF75 = endDF.drop(['name'], axis=1).groupby(['step', 'run'], as_index=False).quantile(.75)
endDF5 = endDF.drop(['name'], axis=1).groupby(['step', 'run'], as_index=False).quantile(.5)
endDF25 = endDF.drop(['name'], axis=1).groupby(['step', 'run'], as_index=False).quantile(.25)
endDFmin = endDF.drop(['name'], axis=1).groupby(['step', 'run'], as_index=False).min()

endDFmax['quantile'] = ['max' for _ in range(len(endDFmax))]
endDF75['quantile'] = ['.75' for _ in range(len(endDF75))]
endDF5['quantile'] = ['.5' for _ in range(len(endDF5))]
endDF25['quantile'] = ['.25' for _ in range(len(endDF25))]
endDFmin['quantile'] = ['min' for _ in range(len(endDFmin))]

endDFquantiled = pd.concat([endDFmax, endDF75, endDF5, endDF25, endDFmin])
endDFquantiled #quantiles per company

In [ ]:
fig = px.scatter(endDFquantiled[endDFquantiled['step'] >0], x="cc", y="tc", color='quantile', title='capabilities',
                 animation_frame="step", template="simple_white")
fig.show()

In [ ]:
dataFrame = endDFsums
x_variable_to_show = dataFrame.groupby(['step'], as_index=False).max()['step']
y_variable_to_show = '# of BNDES financed projects'


fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show,
           y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].max()[y_variable_to_show], 
               marker=dict(color="crimson", size=5), mode="markers"), secondary_y=False
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.75)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.5)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.25)[y_variable_to_show]),
             secondary_y=False)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show,
           y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].min()[y_variable_to_show], 
               marker=dict(color="blue", size=5), mode="markers"),
              secondary_y=False)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show,
           y=endDF.groupby(['step'], as_index=False)['lc'].mean()['lc']),
              secondary_y=True)
# Change the bar mode
fig.update_layout(barmode='overlay', template="simple_white")
fig.show()

In [ ]:
x_variable_to_show = endDF.groupby(['step'], as_index=False).max()['step']
y_variable_to_show = '# of projects yet to be financed'

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show,
           y=endDFsums.groupby(['step'], as_index=False)[y_variable_to_show].max()[y_variable_to_show], 
               marker=dict(color="crimson", size=5), mode="markers"), secondary_y=False
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show,
                     y=endDFsums.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.75)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show,
                     y=endDFsums.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.5)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show,
                     y=endDFsums.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.25)[y_variable_to_show]),
             secondary_y=False)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show,
           y=endDFsums.groupby(['step'], as_index=False)[y_variable_to_show].min()[y_variable_to_show], 
               marker=dict(color="blue", size=5), mode="markers"),
              secondary_y=False)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show,
           y=endDF.groupby(['step'], as_index=False)['lc'].min()['lc']),
              secondary_y=True)
# Change the bar mode
fig.update_layout(barmode='overlay', template="simple_white")
fig.show()

In [ ]:
#cumulative internalisation_index

dataFrame = endDFsums
x_variable_to_show = dataFrame.groupby(['step'], as_index=False).max()['step']
y_variable_to_show = 'internalisation_index'

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show,
           y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].max()[y_variable_to_show], 
               marker=dict(color="crimson", size=5), mode="markers"), secondary_y=False
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.75)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.5)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.25)[y_variable_to_show]),
             secondary_y=False)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show,
           y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].min()[y_variable_to_show], 
               marker=dict(color="blue", size=5), mode="markers"),
              secondary_y=False)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show,
           y=endDF.groupby(['step'], as_index=False)['lc'].min()['lc']),
              secondary_y=True)
# Change the bar mode
fig.update_layout(barmode='overlay', template="simple_white")
fig.show()

In [ ]:
dataFrame = endDF
x_variable_to_show = dataFrame.groupby(['step'], as_index=False).max()['step']
y_variable_to_show = '# of rounds without access to finance'

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(name='max', x=x_variable_to_show,
           y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].max()[y_variable_to_show], 
               marker=dict(color="crimson", size=5), mode="markers"), secondary_y=False
)

fig.add_trace(go.Bar(name='.75 q',
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.75)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.5 q', 
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.5)[y_variable_to_show]),
             secondary_y=False)
fig.add_trace(go.Bar(name='.25 q', 
                     x=x_variable_to_show,
                     y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].quantile(.25)[y_variable_to_show]),
             secondary_y=False)

fig.add_trace(go.Scatter(name='min', x=x_variable_to_show,
           y=dataFrame.groupby(['step'], as_index=False)[y_variable_to_show].min()[y_variable_to_show], 
               marker=dict(color="blue", size=5), mode="markers"),
              secondary_y=False)

fig.add_trace(go.Scatter(name='LC', x=x_variable_to_show,
           y=endDF.groupby(['step'], as_index=False)['lc'].min()['lc']),
              secondary_y=True)
# Change the bar mode
fig.update_layout(barmode='overlay', template="simple_white")
fig.show()